In [1]:
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import linalg
from scipy import linalg
import copy

In [9]:
# This is a test case for computing current flow betweeness of a protein residue network containing 10 amino acid (resid 0-9)
# Step1: allow user to upload a 3-cloumn csv file [residA  residB   weight]
# convert automatically to a np.array and adjacency matrix (see Figure 2 in DOI: 10.1021/acs.jctc.8b01197)
# weight is usualy a pair-wise correlation coefficient or normalized interaction energy between {0, 1} from molecular dynamics simulation


tempAdj=sp.sparse.coo_matrix(
    (
        np.array(
            [.25,.33,.5,1,.25,.25,.33,.33,.5,.5,1,
             1,.25,.25,.33,.33,.5,.5,1,1,.25,.33,.5,1]),
        ([0,0,0,0,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,9,9],
         [1,2,3,4,0,5,0,6,0,7,0,8,1,9,2,9,3,9,4,9,5,6,7,8])
    )
)
tempAdjDense=tempAdj.todense()
print(tempAdj)
print(tempAdjDense)

  (0, 1)	0.25
  (0, 2)	0.33
  (0, 3)	0.5
  (0, 4)	1.0
  (1, 0)	0.25
  (1, 5)	0.25
  (2, 0)	0.33
  (2, 6)	0.33
  (3, 0)	0.5
  (3, 7)	0.5
  (4, 0)	1.0
  (4, 8)	1.0
  (5, 1)	0.25
  (5, 9)	0.25
  (6, 2)	0.33
  (6, 9)	0.33
  (7, 3)	0.5
  (7, 9)	0.5
  (8, 4)	1.0
  (8, 9)	1.0
  (9, 5)	0.25
  (9, 6)	0.33
  (9, 7)	0.5
  (9, 8)	1.0
[[0.   0.25 0.33 0.5  1.   0.   0.   0.   0.   0.  ]
 [0.25 0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.33 0.   0.   0.   0.   0.   0.33 0.   0.   0.  ]
 [0.5  0.   0.   0.   0.   0.   0.   0.5  0.   0.  ]
 [1.   0.   0.   0.   0.   0.   0.   0.   1.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.25]
 [0.   0.   0.33 0.   0.   0.   0.   0.   0.   0.33]
 [0.   0.   0.   0.5  0.   0.   0.   0.   0.   0.5 ]
 [0.   0.   0.   0.   1.   0.   0.   0.   0.   1.  ]
 [0.   0.   0.   0.   0.   0.25 0.33 0.5  1.   0.  ]]


In [10]:
# Step2: convert adjacency matrix to Laplacian matrix using sum of diagonal terms
tempLapDense=-copy.deepcopy(tempAdjDense)
for ii,irow in enumerate(tempLapDense):
  tempLapDense[ii,ii]=-np.sum(irow)

print(tempLapDense)

[[ 2.08 -0.25 -0.33 -0.5  -1.   -0.   -0.   -0.   -0.   -0.  ]
 [-0.25  0.5  -0.   -0.   -0.   -0.25 -0.   -0.   -0.   -0.  ]
 [-0.33 -0.    0.66 -0.   -0.   -0.   -0.33 -0.   -0.   -0.  ]
 [-0.5  -0.   -0.    1.   -0.   -0.   -0.   -0.5  -0.   -0.  ]
 [-1.   -0.   -0.   -0.    2.   -0.   -0.   -0.   -1.   -0.  ]
 [-0.   -0.25 -0.   -0.   -0.    0.5  -0.   -0.   -0.   -0.25]
 [-0.   -0.   -0.33 -0.   -0.   -0.    0.66 -0.   -0.   -0.33]
 [-0.   -0.   -0.   -0.5  -0.   -0.   -0.    1.   -0.   -0.5 ]
 [-0.   -0.   -0.   -0.   -1.   -0.   -0.   -0.    2.   -1.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.25 -0.33 -0.5  -1.    2.08]]


In [11]:
#Step3: compute pseudoinverse of Laplacian matrix using np.linalg.pinv()
tempLinv=np.linalg.pinv(tempLapDense)
tempLinv

matrix([[ 0.56118298, -0.07920163,  0.01776807,  0.12079837,  0.22079837,
         -0.31958625, -0.22261655, -0.11958625, -0.01958625, -0.15997086],
        [-0.07920163,  2.10733683, -0.46236014, -0.35932984, -0.25932984,
          0.69387529, -0.54248834, -0.43945804, -0.33945804, -0.31958625],
        [ 0.01776807, -0.46236014,  1.65481158, -0.26236014, -0.16236014,
         -0.54248834,  0.56458236, -0.34248834, -0.24248834, -0.22261655],
        [ 0.12079837, -0.35932984, -0.26236014,  1.1740035 , -0.05932984,
         -0.43945804, -0.34248834,  0.42720862, -0.13945804, -0.11958625],
        [ 0.22079837, -0.25932984, -0.16236014, -0.05932984,  0.70733683,
         -0.33945804, -0.24248834, -0.13945804,  0.29387529, -0.01958625],
        [-0.31958625,  0.69387529, -0.54248834, -0.43945804, -0.33945804,
          2.10733683, -0.46236014, -0.35932984, -0.25932984, -0.07920163],
        [-0.22261655, -0.54248834,  0.56458236, -0.34248834, -0.24248834,
         -0.46236014,  1.6548115

In [12]:
# Step4: compute flow betweeness for edge 4-8
v_1_10_4=tempLinv[3,0]-tempLinv[3,9]
v_1_10_8=tempLinv[7,0]-tempLinv[7,9]
b_4_8=tempAdjDense[3,7]*(v_1_10_4-v_1_10_8)

print('v_1_10_4',v_1_10_4)
print('v_1_10_8',v_1_10_8)
print('b_4_8',b_4_8)

v_1_10_4 0.24038461538461647
v_1_10_8 -0.2403846153846146
b_4_8 0.24038461538461553


In [13]:
# or a concise way of computing
tempAdjDense[3,7]*np.matrix(np.atleast_2d(np.array([0,0,0,1,0,0,0,-1,0,0])))*tempLinv*np.matrix(np.atleast_2d(np.array([1,0,0,0,0,0,0,0,0,-1])).T)

matrix([[0.24038462]])

In [ ]:
# Now, can you build a web interface that allow user to import a csv file (number of residues < 100 for testing) and resid of source and sink, obtain the betweenness score for each edge?

In [15]:
# Alternatively, if user only needs to compute flow betweenness for one pair of source and sink, conjugate gradient is faster

#sources=[0,1,2]
#sinks=[5,8,9]
sources=[0]
sinks=[9]
loadMat=np.zeros((len(tempLinv),len(sources)*len(sinks)))
ii=0
for isource,source in enumerate(sources):
  for isink,sink in enumerate(sinks):
    loadMat[source,ii]=1
    loadMat[sink,ii]=-1
    ii+=1
print('Load mat',loadMat)

# this can be done using conjugate gradient! #
potMat=tempLinv*loadMat
print('pot mat',potMat)
potVec=np.sum(potMat,axis=1)
print('pot vec',potVec)
# #
#assuming you have a cg_solver method that takes in
#a sparse matrix and a vector as input
#you would use something like:

#potVec=np.zeros(tempAdj.shape[0])
#for loadVec in loadMat.T:
#  potVec=potVec+sp.sparse.linalg.cg(tempLapSp,loadVec)[0]
#potVec=potVec/len(loadMat.T)


btw_4_8=np.abs(tempAdj.tocsr()[3,7]*(potVec[7]-potVec[3]))
print('btw_4_8',btw_4_8)

Load mat [[ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]]
pot mat [[ 0.72115385]
 [ 0.24038462]
 [ 0.24038462]
 [ 0.24038462]
 [ 0.24038462]
 [-0.24038462]
 [-0.24038462]
 [-0.24038462]
 [-0.24038462]
 [-0.72115385]]
pot vec [[ 0.72115385]
 [ 0.24038462]
 [ 0.24038462]
 [ 0.24038462]
 [ 0.24038462]
 [-0.24038462]
 [-0.24038462]
 [-0.24038462]
 [-0.24038462]
 [-0.72115385]]
btw_4_8 [[0.24038462]]
